In [4]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 28 22:59:51 2022

@author: unily
"""
import os
import pandas as pd
import difflib
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import string
import plotly.express as px
import numpy as np
from plotly.offline import plot, init_notebook_mode, iplot

punctuations = string.punctuation
path = r'C:\Users\unily\Documents\Georgetown\Data Science II App Stat Lng\final_project'
os.chdir(path)
# Load master data
df = pd.read_csv(r'data\walmart_ni_2022-05-06.csv')
#Load INIDE's Canasta Basica estimates for march 2022
basket = pd.read_csv(r'data\basket_march_22.csv', encoding='latin-1')

# Load basic consumption basket original spanish names
lista = pd.read_excel(r'data\canasta.xlsx')
lista['Item']  = lista["Item"].str.replace('de', '')
lista['Item']  = lista["Item"].str.replace('/', '')
# Remove spanish accents
lista['Item'] = lista['Item'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
lista_1 = lista['Item'].str.lower().str.strip()
# Kilogram to pounds
df['weight_lb'] = df['weight'].astype(float)*2.20462
# Price per pound
df['price_lb'] = df['price']/df['weight_lb']

lista_2 = '|'.join(lista_1)
df['name'] = df['name'].str.strip()
df['match'] = df['name'].str.contains(lista_2).fillna(0).astype('int')

#Keep only data gathered starting from april 2022, since it has all categories of grocery items
df_2 = df[df['match']==True].copy()
df_2['mes'] = pd.to_datetime(df['date'], errors= 'coerce').dt.month
df_2 = df_2[df_2['mes']>=4]
#Specific String finder
#df['string'] = df['name'].str.extract('(pasta)')

# List of identified item names that are comparable in price-weight with those in the official basic consumption basket
canasta = ['arroz el faisan', 'frijol sabemasanel rojo', 'azucar san antonio regular',
           'aceite rico vegetal', 'carne de res para asar', 'posta de cerdo empacado',
           'pechuga entera pollo fresca', 'torta de pescado caf bolsa', 
           'leche veritos pinolillo bolsa', 'media cajilla de huevo blanco barranca  unidades',
           'queso monte fresco seco', 'tortilla milpa real tostadas planas',
           'pinolillo sabemas', 'pasta sabemas spaguetti gr', 'harina bollo fino de trigo',
           'pasta dental colgate triple acción', 'fosforos suli de madera   cajitas',
           'toalla sanitaria kotex esencial sin alas aloe   unidades', 'desodorante barra rexona men invisible',
           'cepillo de bolsillo equate con espejo  unidad', 'pack jabon dove de tocador original']
canasta_2 = '|'.join(canasta)

df['match_canasta'] = df['name'].str.contains(canasta_2).fillna(0).astype('int')
df_work = df[df['match_canasta']==True].copy()

#Merge to get official weight for each item
df_work = df_work.merge(basket, how="left", on="name")

# Treemap graph

fig2 = px.treemap(basket, path=[px.Constant("Basket"), 'Subtype', 'Item'], values='value',
                  color='value', hover_data=['Subtype'],
                  color_continuous_scale='RdBu',template="simple_white",
                  title= '')
fig2.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig2.update_layout(showlegend=True, legend_title_text='Monthly value in Córdobas (C$)',
                xaxis_title="", 
                yaxis_title="",
                font_family="Arial", title_font_family="Arial Black",
                yaxis_title_font_family ="Arial Black",
                xaxis_title_font_family ="Arial Black",
                title_font_color="black",
                title_font_size=19, legend_title_font_color="black",
                legend=dict(yanchor="top", y=0.89, xanchor="right", x=1),
                )
fig2.write_image("assets/fig2.png", width=800, height=650)
iplot(fig2)

# Get average total cost per month
df_work['monthly_cost'] = df_work['price_lb'] * df_work['Quantity']
df_work['monthly_cost'] = df_work['monthly_cost'].fillna(df_work['price'] * df_work['Quantity'])

#Unitary prices for the following items
df_work.loc[df_work['name'] == 'desodorante barra rexona men invisible', 'monthly_cost'] = df_work['price'] * df_work['Quantity']
df_work.loc[df_work['name'] == 'tortilla diana de maiz jalapeños', 'monthly_cost'] = df_work['price'] * df_work['Quantity']
df_work.loc[df_work['name'] == 'fosforos suli de madera   cajitas', 'monthly_cost'] = df_work['price'] # amount the same as in official consumption estimate
df_work.loc[df_work['name'] == 'pasta dental colgate triple acción', 'monthly_cost'] = df_work['price'] # amount the same as in official consumption estimate
df_work.loc[df_work['name'] == 'pack jabon dove de tocador original', 'monthly_cost'] = df_work['price'] # amount the same as in official consumption estimate

df_work.loc[df_work['name'] == 'tortilla milpa real tostadas planas', 'monthly_cost'] = (df_work['price_lb'] * df_work['Quantity'] ) * 0.25
# Cost multiplied by a factor of 1.5 to account for the fact it is not bread but flour 
df_work.loc[df_work['name'] == 'harina bollo fino de trigo', 'monthly_cost'] = (df_work['price_lb'] * df_work['Quantity'] ) * 1.5
# Cost multiplied by a factor of 10 to account for the fac bulk price is much cheaper!
df_work.loc[df_work['name'] == 'azucar san antonio regular', 'monthly_cost'] = (df_work['price_lb'] * df_work['Quantity'] ) * 1
# Take price and add 2 more liters to reach consumption quantity
df_work.loc[df_work['name'] == 'aceite rico vegetal', 'monthly_cost'] = df_work['price'] + ((df_work['Quantity'] - df_work['weight'] ) * df_work['price_lb'])

# Monthly cost summaries
# Fill NAs in the weight of some items
df_work['weight'] = df_work['weight'].fillna(1)
df_sum = df_work.groupby([pd.to_datetime(df_work.date), df_work.name, 
                          df_work.weight, df_work.Item])['monthly_cost'].mean().\
                            reset_index().rename(columns={0:'mean'})
df_sum = df_sum.sort_values(['date', 'name', 'monthly_cost'], ascending =False) # sort values
df_sum = df_sum.drop_duplicates(['date','name'], keep='first') # drop item duplicates
# Generate average montlhy costs
df_sum = df_sum.groupby([pd.to_datetime(df_sum.date).dt.month, df_sum.name])['monthly_cost'].mean().\
                            reset_index().rename(columns={0:'mean'})
#Keep one value per month, excluding may for which there is scarce data still
df_sum['month'] =  df_sum['date'].copy()
df_sum = df_sum.loc[df_sum['date']!=5] # Exclude May because there is little data
df_sum['date'] = df_sum['date'].replace({1: 'January', 2: 'February', 3: 'March',
      4: 'April'})

# Graph 2
#Merge to get names in English
df_sum = df_sum.merge(basket, how="left", on="name")
#df_sum = df_sum.sort_values('monthly_cost', ascending=False)
df_sum = df_sum.sort_values(['month', 'monthly_cost'], ascending=True, ignore_index =True)

fig3 = px.bar(df_sum, x="date", y="monthly_cost", color="Item", 
              title="",
              text = df_sum["monthly_cost"].astype('int'),
                  color_discrete_sequence=px.colors.qualitative.Pastel, template="simple_white")

fig3.update_layout(showlegend=True, legend_title_text='Items',
                xaxis_title="Month of 2022", 
                yaxis_title="Monthly value in Córdobas (C$)",
                font_family="Arial", title_font_family="Arial Black",
                yaxis_title_font_family ="Arial Black",
                xaxis_title_font_family ="Arial Black",
                title_font_color="black",
                title_font_size=19, legend_title_font_color="black",
                legend=dict(yanchor="top", y=0.89, xanchor="left", x=1),
                margin = dict(t=50, l=25, r=25, b=25))
fig3.update_traces(textfont_size=12, textangle=0, textposition="inside", cliponaxis=False)
fig3.write_image("assets/fig3.png", width=800, height=650)
iplot(fig3)

# Graph 3
# Load Official CPI data
cpi = pd.read_csv(r'data\cpi_official.csv')

cpi['CPI_text']  = cpi.CPI.copy()
cpi.CPI  = cpi.CPI.str.replace('%', '')
cpi.CPI = pd.to_numeric(cpi.CPI)

cpi['CPI_Food_text']  = cpi.CPI_Food.copy()
cpi.CPI_Food  = cpi.CPI_Food.str.replace('%', '')
cpi.CPI_Food = pd.to_numeric(cpi.CPI_Food)
cpi.rename(columns={'CPI': 'CPI_General'}, inplace = True) # rename CPI

fig1 = px.line(cpi, x='date', y=cpi.columns[1:3], 
                  title = '',
               template ='simple_white'
                 )
fig1.update_layout(showlegend=True, legend_title_text='CPI - component',
                xaxis_title="Date",
                yaxis_title="Interannual inflation rate (%)",
                font_family="Arial",
                title_font_family="Arial Black",
                yaxis_title_font_family ="Arial Black",
                xaxis_title_font_family ="Arial Black",
                title_font_color="black",
                title_font_size=19,
                legend_title_font_color="black",
                uniformtext_minsize=14, 
                uniformtext_mode=False)
fig1.update_traces(textposition='middle right')
fig1.update_xaxes(showline=True, linewidth=2, linecolor='black', showspikes=True)
fig1.update_yaxes(showline=True, linewidth=2, linecolor='black', showspikes=True, range=[0, 15])
#### Display figure
fig1.write_image("assets/fig1.png", width=800, height=650)
iplot(fig1)

## Additional Graphs
# Graph 4
fig4 = px.bar(df[0:15].sort_values('price'), x='price', y="name", 
             template="simple_white", text='price',
             title= '')

fig4.update_layout(showlegend=True, legend_title_text='',
                xaxis_title="Price in Local Currency Units (nominal C$)", 
                yaxis_title="",
                font_family="Arial", title_font_family="Arial Black",
                yaxis_title_font_family ="Arial Black",
                xaxis_title_font_family ="Arial Black",
                title_font_color="black",
                title_font_size=19, legend_title_font_color="black",
                legend=dict(yanchor="top", y=0.89, xanchor="right", x=1),
                )
fig4.update_traces(textposition='inside')
fig4.add_annotation(
    text="Source: author.",
    xref="x domain", yref="y domain",x=0.01, y=-0.14,showarrow=False)
fig4.write_image("assets/fig4.png", width=800, height=650)
iplot(fig4)


# Working drafts of text similatiry libraries below
'''
df_2['name_2'] = df_2['name']
df_2['match_2'] = [(len(difflib.get_close_matches(x, df_2['name_2'], cutoff=0.5))>1)*1 
              for x in df_2['name_2']]
df_2 = df_2[df_2['match_2']==1]
df_2 = df_2.sort_values('conteo', ascending=False).drop_duplicates('name')
df_2.assign(Output=[process.extract(i, df_2['name'], limit=3) for i in lista['Unnamed: 0']])
lista_2 = df_2['name'].tolist()
fuz = []
for i in lista_1:
    fuz.append(process.extract(i, lista_2, limit=4))
df_2['matches'] = fuz
dframe1.show()
'''


C:\Users\unily\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:41: UserWarning:

This pattern has match groups. To actually get the groups, use str.extract.



"\ndf_2['name_2'] = df_2['name']\ndf_2['match_2'] = [(len(difflib.get_close_matches(x, df_2['name_2'], cutoff=0.5))>1)*1 \n              for x in df_2['name_2']]\ndf_2 = df_2[df_2['match_2']==1]\ndf_2 = df_2.sort_values('conteo', ascending=False).drop_duplicates('name')\ndf_2.assign(Output=[process.extract(i, df_2['name'], limit=3) for i in lista['Unnamed: 0']])\nlista_2 = df_2['name'].tolist()\nfuz = []\nfor i in lista_1:\n    fuz.append(process.extract(i, lista_2, limit=4))\ndf_2['matches'] = fuz\ndframe1.show()\n"